<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/vector_embeddings/examples_faiss_in_memory_vector_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ollama langchain_community --quiet
!pip install openai --quiet
!pip install faiss-cpu --quiet

host="localhost:11434"
embedding_model = "mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {embedding_model}")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.7 MB/s eta 0:00:00


In [2]:
get_ipython().system_raw(f"ollama pull {embedding_model}")


In [3]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [4]:
def embed(text: str) -> list[float]:
  return llm.embeddings.create(input = [text], model=embedding_model).data[0].embedding

def createEmbeddings(texts: list[str]) -> list[list[float]]:
  return [embed(txt) for txt in texts]


In [5]:
import faiss
import numpy as np

# Sample sentences
sentences = [
    "Artificial Intelligence is the future.",
    "AI requires large datasets to train models.",
    "Machines learn by analyzing data."
]

# Convert to embeddings
embeddings = createEmbeddings(sentences)

# Store in FAISS index for similarity search
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [8]:
# Query the most similar sentence
query = embed("AI evolves with data.")
D, I = index.search(np.array([query]), 3)
print(f"The most similar sentence is: {sentences[I[0][0]]}")

The most similar sentence is: AI requires large datasets to train models.


In [9]:
print (D)

[[0.45778984 0.51349163 0.5334135 ]]


In [14]:
# range queries
query = embed("AI evolves with data.")

distance_threshold=0.52
lims, D, I  = index.range_search(np.array([query]), distance_threshold)
indexes=I[lims[0]:lims[0+1]]
distances=D[lims[0]:lims[0+1]]

print (indexes)
print (distances)

[1 2]
[0.45778984 0.51349163]
